<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/stock/labelling_stock_selection2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('/content/nifty_50_weekly_data.csv')
# Convert the date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Sort the dataframe by ticker and date
df = df.sort_values(['ticker', 'date'])

# Group the dataframe by ticker
grouped = df.groupby('ticker')

# Function to calculate weekly returns
def calculate_weekly_return(group):
    # Shift the 'adj close' column by 1 day (1 month)
    group['next_week_close'] = group['adj close'].shift(-1)

    # Calculate the weekly return
    group['weekly_return'] = (group['next_week_close'] - group['adj close']) / group['adj close']

    return group

# Apply the function to each group (ticker)
df_with_returns = grouped.apply(calculate_weekly_return)

# Reset the index
df_with_returns = df_with_returns.reset_index(drop=True)

# Remove rows where we couldn't calculate the weekly return (last week of data for each stock)
df_with_returns = df_with_returns.dropna(subset=['weekly_return'])

# Display the first few rows of the result
print(df_with_returns[['date', 'ticker', 'adj close', 'next_week_close', 'weekly_return']].head())

# Calculate summary statistics of weekly returns for each stock
weekly_return_summary = df_with_returns.groupby('ticker')['weekly_return'].agg(['mean', 'median', 'std', 'min', 'max'])
print("\nWeekly Return Summary Statistics:")
print(weekly_return_summary)

# Find the stock with the highest average weekly return
best_performing_stock = weekly_return_summary['mean'].idxmax()
print(f"\nBest performing stock based on average weekly return: {best_performing_stock}")

# Save the results to a CSV file
df_with_returns.to_csv('stock_weekly_returns.csv', index=False)
print("\nResults saved to 'stock_weekly_returns.csv'")

        date       ticker    adj close  next_week_close  weekly_return
0 2021-05-09  ADANIENT.NS  1295.667725      1217.313232      -0.060474
1 2021-05-16  ADANIENT.NS  1217.313232      1311.827393       0.077642
2 2021-05-23  ADANIENT.NS  1311.827393      1299.358398      -0.009505
3 2021-05-30  ADANIENT.NS  1299.358398      1696.717163       0.305812
4 2021-06-06  ADANIENT.NS  1696.717163      1597.614380      -0.058409

Weekly Return Summary Statistics:
                   mean    median       std       min       max
ticker                                                         
ADANIENT.NS    0.013131  0.013128  0.062439 -0.086110  0.305812
ADANIPORTS.NS  0.002347  0.007076  0.050644 -0.173147  0.098527
APOLLOHOSP.NS  0.005770  0.011657  0.055117 -0.140761  0.196831
ASIANPAINT.NS  0.005961  0.013105  0.045238 -0.122163  0.099447
AXISBANK.NS    0.001075 -0.002327  0.040433 -0.091466  0.103519
BAJAJ-AUTO.NS  0.002192  0.000365  0.032739 -0.075903  0.071905
BAJAJFINSV.NS  0.005734  0.